In [1]:
import json
import xml.etree.cElementTree as ET

In [2]:
fs = open('../data/fairsharing_20230131.json')
lines = fs.read().split('\n')

In [3]:

def fairsharingMap(line):
    entry = ET.Element("registryEntry")
    dic = json.loads(line)
    attributes = dic['attributes']
    metadata = attributes['metadata']
    if 'doi' in metadata and metadata['doi'] != "":
        ET.SubElement(entry, "identifier", type = "doi").text = metadata['doi']
    if 'identifier' in metadata and metadata['identifier'] != "":
        ET.SubElement(entry, 'internalIdentifier').text = str(metadata['identifier'])
    if 'name' in metadata and metadata['name'] != '':
        ET.SubElement(entry,'name').text = metadata['name']
    if 'abbreviation' in metadata and metadata['abbreviation'] != "":
        ET.SubElement(entry,'additionalName').text = metadata['abbreviation']
    if 'homepage' in metadata and metadata['homepage'] != "":
        ET.SubElement(entry,'url').text = metadata['homepage']
    if 'description' in metadata and metadata['description'] != '':
        ET.SubElement(entry,'description').text = metadata['description'].strip()
    if 'subjects' in attributes:
        for sub in attributes['subjects'] :
            ET.SubElement(entry,'subject').text = sub
    if 'user_defined_tags' in attributes:
        for kw in attributes['user_defined_tags'] :
            ET.SubElement(entry,'keyword').text = kw
    if 'grants' in attributes and attributes['grants'] != []:
        for g in attributes['grants']:
            if g['saved_state']['name'] != '':
                organization = ET.Element("organization")
                ET.SubElement(organization,"name").text = g['saved_state']['name']
                if 'types' in g['saved_state'] and g['saved_state']['types'] != []:
                    ET.SubElement(organization,"type").text = g['saved_state']['types'][0]
                if 'relation' in g['saved_state'] and g['saved_state']['relation'] != '':
                    ET.SubElement(organization, "role").text = g['saved_state']['relation']
                entry.append(organization) #ET.SubElement(entry, organization)
    if 'data_versioning' in metadata and metadata['data_versioning'] != '' and metadata['data_versioning'] == 'yes':
        ET.SubElement(entry, "versioning").text = "True" 
    else :
        ET.SubElement(entry, "versioning").text = "False" 
    if 'data_access_condition' in metadata and metadata['data_access_condition'] != {} and 'type' in metadata['data_access_condition']:
        ac = metadata['data_access_condition']
        if 'url' in ac and 'notes' in ac:
            ET.SubElement(entry, "access", type=ac['type'] , url = ac['url'], restriction = ac['notes']).text = "dataAccess"
        elif 'url' in ac:
            ET.SubElement(entry, "access", type=ac['type'] , url = ac['url']).text = "dataAccess"
        elif 'notes' in ac:
            ET.SubElement(entry, "access", type=ac['type'] ,restriction = ac['notes']).text = "dataAccess"
        else:
            ET.SubElement(entry, "access", type=ac['type']).text = "dataAccess"
    if 'data_deposition_condition' in metadata and metadata['data_deposition_condition'] != {} and 'type' in metadata['data_deposition_condition']:
        ac = metadata['data_deposition_condition']
        if 'url' in ac and 'notes' in ac:
            ET.SubElement(entry, "access", type=ac['type'] , url = ac['url'], restriction = ac['notes']).text = "dataDeposition"
        elif 'url' in ac:
            ET.SubElement(entry, "access", type=ac['type'] , url = ac['url']).text = "dataDeposition"
        elif 'notes' in ac:
            ET.SubElement(entry, "access", type=ac['type'] ,restriction = ac['notes']).text = "dataDeposition"
        else:
            ET.SubElement(entry, "access", type=ac['type'] ).text = "dataDeposition"
    if 'licence_links' in attributes and attributes['licence_links'] != []:
        for lic in attributes['licence_links']:
            if 'licence_name' in lic and lic['licence_name'] != '' and 'licence_url' in lic and lic['licence_url'] != '':
                ET.SubElement(entry, "licence", type = lic['licence_name'], url = lic['licence_url']).text = "databaseAccess"
    if 'year_creation' in metadata and metadata['year_creation'] != '':
        ET.SubElement(entry, 'startDate', format = "YYYY").text = str(metadata['year_creation'])
    return entry
    
    

In [4]:
entry = ET.Element("registryEntries")

for line in lines:
    if line == "":
        continue
    entry.append(fairsharingMap(line))

fout = open("../commonModel/fs.xml",'w')
tree = ET.ElementTree(entry)
tree.write("../commonModel/fs.xml")